In [1]:
import os
import sys
import time

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import psutil
import seaborn as sns
import torch
import torch.optim as optim
from dotenv import load_dotenv
from gensim.models import FastText, Word2Vec
from sklearn.decomposition import LatentDirichletAllocation, NMF, PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer, StandardScaler
from sklearn.manifold import TSNE
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec
from torch_geometric.utils import from_networkx
from sentence_transformers import SentenceTransformer

In [3]:
# Загрузка переменных окружения из файла .env
load_dotenv()

True

In [4]:
# Получение текущей директории
current_dir = os.getcwd()
# Получение корневой директории проекта
project_root = os.path.dirname(os.path.dirname(current_dir))

In [5]:
# Добавление корневой директории проекта в sys.path для импорта модулей
if project_root not in sys.path:
    sys.path.append(project_root)

In [6]:
# Пути к файлам с исходными данными
df_raw_json_path = os.path.join(project_root, 'data', 'raw', 'steam_games_data.json')
df_raw_csv_path = os.path.join(project_root, 'data', 'raw', 'steam_games_data.csv')

In [7]:
# Пути к файлам с обработанными данными
df_processed_json_path = os.path.join(project_root, 'data', 'processed', 'steam_games_data.json')
df_processed_csv_path = os.path.join(project_root, 'data', 'processed', 'steam_games_data.csv')

In [8]:
# Определение устройства для работы с Torch (GPU при наличии, иначе CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [9]:
# Загрузка обработанных данных
print("Загрузка данных...")
df = pd.read_json(df_processed_json_path)
print("Данные загружены.")

Загрузка данных...
Данные загружены.


In [10]:
# Разделение данных на обучающую и тестовую выборки
print("Разделение данных на обучающую и тестовую выборки...")
train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)
print("Данные разделены.")

Разделение данных на обучающую и тестовую выборки...
Данные разделены.


In [11]:
def reduce_dataset(df, percentage=0.1):
    """
    Уменьшает размер датасета до указанного процента.

    Args:
        df (pd.DataFrame): Исходный DataFrame.
        percentage (float): Процент данных для сохранения (от 0 до 1).

    Returns:
        pd.DataFrame: Уменьшенный DataFrame.
    """
    if not 0 <= percentage <= 1:
        raise ValueError("Процент должен быть в диапазоне от 0 до 1")

    print(f"Уменьшение датасета до {percentage * 100}%...")

    # Сортировка по столбцу 'estimated_owners' в порядке убывания
    df_sorted = df.sort_values(by='estimated_owners', ascending=False)

    # Вычисление количества строк для сохранения
    num_rows = int(len(df_sorted) * percentage)

    # Выборка первых строк
    reduced_df = df_sorted.head(num_rows)

    print(f"Датасет уменьшен до {len(reduced_df)} строк.")

    return reduced_df

In [12]:
# df = reduce_dataset(df, percentage=0.1)

In [13]:
df.shape

(51865, 16)

In [14]:
def vectorize_owners(df, method='log_scale'):
    """
    Векторизует данные о владельцах игр.

    Args:
        df (pd.DataFrame): DataFrame с данными об играх.
        method (str): Метод векторизации ('log_scale' или 'standard').

    Returns:
        np.ndarray: Векторизованные данные о владельцах.
    """
    print(f"Начало векторизации владельцев с использованием метода: {method}")
    owners = df['estimated_owners'].values.reshape(-1, 1)
    if method == 'log_scale':
        owners = np.log1p(owners)
        scaler = MinMaxScaler()
        owners = scaler.fit_transform(owners)
        # Усиление значений для игр с большим количеством владельцев
        owners_weighted = owners * (1 + (owners * 2))  # Пример: нелинейное усиление
        return owners_weighted
    elif method == 'standard':
        scaler = StandardScaler()
        owners = scaler.fit_transform(owners)
        # Можно также применить нелинейное преобразование после стандартизации
        return owners
    print(f"Завершение векторизации владельцев с использованием метода: {method}")
    return owners

In [15]:
def vectorize_tags(df, method='tfidf', vectorizer=None, word2vec_params=None, fasttext_params=None, multilabel_params=None):
    """
    Векторизует теги игр различными методами.

    Args:
        df (pd.DataFrame): DataFrame с данными об играх.
        method (str): Метод векторизации ('tfidf', 'multilabel', 'word2vec', 'fasttext', 'node2vec').
        vectorizer: Объект векторизатора для TF-IDF.
        word2vec_params (dict): Параметры для Word2Vec.
        fasttext_params (dict): Параметры для FastText.
        multilabel_params (dict): Параметры для MultiLabelBinarizer.

    Returns:
        tuple: Векторизованные теги и модель (или векторизатор).
    """
    print(f"Начало векторизации тегов с использованием метода: {method}")
    tags = df['all_tags'].apply(lambda x: ' '.join(x))
    if method == 'tfidf':
        if vectorizer is None:
            vectorizer = TfidfVectorizer()
            tags_vectorized = vectorizer.fit_transform(tags).toarray()
        else:
            tags_vectorized = vectorizer.transform(tags).toarray()
        print(f"Завершение векторизации тегов с использованием метода: {method}")
        return tags_vectorized, vectorizer
    elif method == 'multilabel':
        default_params = {'sparse_output': False}
        params = multilabel_params if multilabel_params else default_params
        print(f"Параметры MultiLabelBinarizer: {params}")
        mlb = MultiLabelBinarizer(**params)
        mlb.fit(df['all_tags'])
        tags_vectorized = mlb.transform(df['all_tags'])
        print(f"Завершение векторизации тегов с использованием метода: {method}")
        return tags_vectorized, mlb
    elif method == 'word2vec':
        sentences = df['all_tags'].tolist()
        default_params = {'vector_size': 100, 'window': 5, 'min_count': 5, 'sg': 1, 'epochs': 10,
                          'workers': psutil.cpu_count(logical=False)}
        params = word2vec_params if word2vec_params else default_params
        print(f"Параметры Word2Vec: {params}")
        model = Word2Vec(sentences=sentences, **params)
        tags_vectorized = np.array(
            [np.mean([model.wv[word] for word in tag if word in model.wv], axis=0) for tag in sentences])
        print(f"Завершение векторизации тегов с использованием метода: {method}")
        return tags_vectorized, model
    elif method == 'fasttext':
        sentences = df['all_tags'].tolist()
        default_params = {'vector_size': 100, 'window': 5, 'min_count': 5, 'sg': 1, 'epochs': 10, 'min_n': 3,
                          'max_n': 6, 'workers': psutil.cpu_count(logical=False)}
        params = fasttext_params if fasttext_params else default_params
        print(f"Параметры FastText: {params}")
        model = FastText(sentences=sentences, **params)
        tags_vectorized = np.array(
            [np.mean([model.wv[word] for word in tag if word in model.wv], axis=0) for tag in sentences])
        print(f"Завершение векторизации тегов с использованием метода: {method}")
        return tags_vectorized, model
    elif method == 'node2vec':
        # Создание графа и обучение Node2Vec выполняется в функции evaluate_vectors
        print(f"Завершение векторизации тегов с использованием метода: {method}")
        return None, None  # Возвращаем None, чтобы обработать в evaluate_vectors
    else:
        raise ValueError("Недопустимый метод векторизации тегов.")

In [16]:
def vectorize_descriptions(df, method='tfidf', vectorizer=None, word2vec_params=None, sentence_bert_model=None,
                           lda_params=None, nmf_params=None):
    """
    Векторизует описания игр различными методами.

    Args:
        df (pd.DataFrame): DataFrame с данными об играх.
        method (str): Метод векторизации ('tfidf', 'word2vec', 'sentence_bert', 'lda', 'nmf').
        vectorizer: Объект векторизатора для TF-IDF.
        word2vec_params (dict): Параметры для Word2Vec.
        sentence_bert_model: Модель Sentence-BERT.
        lda_params (dict): Параметры для LDA.
        nmf_params (dict): Параметры для NMF.

    Returns:
        tuple: Векторизованные описания и модель (или векторизатор).
    """
    print(f"Начало векторизации описаний с использованием метода: {method}")
    if method == 'tfidf':
        if vectorizer is None:
            vectorizer = TfidfVectorizer()
            desc_vectorized = vectorizer.fit_transform(df['short_description_clean']).toarray()
        else:
            desc_vectorized = vectorizer.transform(df['short_description_clean']).toarray()
        print(f"Завершение векторизации описаний с использованием метода: {method}")
        return desc_vectorized, vectorizer
    elif method == 'word2vec':
        sentences = df['short_description_clean'].apply(lambda x: x.split()).tolist()
        default_params = {'vector_size': 100, 'window': 5, 'min_count': 5, 'sg': 0, 'epochs': 10,
                          'workers': psutil.cpu_count(logical=False)}
        params = word2vec_params if word2vec_params else default_params
        print(f"Параметры Word2Vec: {params}")
        model = Word2Vec(sentences=sentences, **params)
        desc_vectorized = np.array([
            np.mean([model.wv[word] for word in desc if word in model.wv], axis=0)
            if any(word in model.wv for word in desc)  # Проверка наличия хотя бы одного слова в словаре
            else np.zeros(model.vector_size)  # Возврат нулевого вектора, если нет слов в словаре
            for desc in sentences
        ])
        print(f"Завершение векторизации описаний с использованием метода: {method}")
        return desc_vectorized, model
    elif method == 'sentence_bert':
        model = sentence_bert_model if sentence_bert_model is not None else SentenceTransformer('all-mpnet-base-v2').to(
            device)
        desc_vectorized = model.encode(df['short_description_clean'].tolist(), device=device)
        print(f"Завершение векторизации описаний с использованием метода: {method}")
        return desc_vectorized, model
    elif method == 'lda':
        if vectorizer is None:
            vectorizer = TfidfVectorizer()
            desc_vectorized = vectorizer.fit_transform(df['short_description_clean'])
        else:
            desc_vectorized = vectorizer.transform(df['short_description_clean'])
        default_params = {'n_components': 10, 'random_state': 0}
        params = lda_params if lda_params else default_params
        print(f"Параметры LDA: {params}")
        lda = LatentDirichletAllocation(**params)
        lda_vectorized = lda.fit_transform(desc_vectorized)
        print(f"Завершение векторизации описаний с использованием метода: {method}")
        return lda_vectorized, (lda, vectorizer)
    elif method == 'nmf':
        if vectorizer is None:
            vectorizer = TfidfVectorizer()
            desc_vectorized = vectorizer.fit_transform(df['short_description_clean'])
        else:
            desc_vectorized = vectorizer.transform(df['short_description_clean'])
        default_params = {'n_components': 10, 'init': 'nndsvdar', 'solver': 'mu', 'beta_loss': 'frobenius',
                          'random_state': 0}
        params = nmf_params if nmf_params else default_params
        print(f"Параметры NMF: {params}")
        nmf = NMF(**params)
        nmf_vectorized = nmf.fit_transform(desc_vectorized)
        print(f"Завершение векторизации описаний с использованием метода: {method}")
        return nmf_vectorized, (nmf, vectorizer)
    else:
        raise ValueError("Недопустимый метод векторизации описаний.")

In [17]:
def evaluate_vectors(train_df, owners_method='log_scale', tags_method='tfidf', desc_method='tfidf', node2vec_model=None,
                     owners_weight_factor=1.0,
                     word2vec_params_tags=None, fasttext_params_tags=None, multilabel_params=None,
                     word2vec_params_desc=None, lda_params=None, nmf_params=None,
                     sentence_bert_model=None):
    """
    Вычисляет и объединяет векторы признаков для игр.

    Args:
        train_df (pd.DataFrame): Обучающий DataFrame.
        owners_method (str): Метод векторизации владельцев.
        tags_method (str): Метод векторизации тегов.
        desc_method (str): Метод векторизации описаний.
        node2vec_model: Обученная модель Node2Vec.
        owners_weight_factor (float): Весовой коэффициент для векторов владельцев.
        word2vec_params_tags (dict): Параметры для Word2Vec (теги).
        fasttext_params_tags (dict): Параметры для FastText (теги).
        multilabel_params (dict): Параметры для MultiLabelBinarizer.
        word2vec_params_desc (dict): Параметры для Word2Vec (описания).
        lda_params (dict): Параметры для LDA.
        nmf_params (dict): Параметры для NMF.
        sentence_bert_model: Модель Sentence-BERT.

    Returns:
        tuple: Объединенные векторы, матрица схожести, скалер, модели векторизации, PCA.
    """
    print(f"Начало evaluate_vectors с владельцами: {owners_method}, тегами: {tags_method}, описаниями: {desc_method}")
    start_time = time.time()

    # Векторизация данных о владельцах
    owners_start_time = time.time()
    owners_vectors = vectorize_owners(train_df, method=owners_method)
    owners_vectors_weighted = owners_vectors * owners_weight_factor
    print(f"Применен весовой коэффициент {owners_weight_factor} к вектору владельцев.")
    owners_end_time = time.time()
    print(f"Векторизация владельцев завершена за {owners_end_time - owners_start_time:.2f} секунд")

    # Векторизация тегов
    tags_start_time = time.time()
    tags_model = None
    tags_vectorizer = None
    if tags_method == 'node2vec':
        if node2vec_model is not None:
            with torch.no_grad():
                tags_vectors_full = node2vec_model().cpu()
            node_id_to_tag = {i: node for i, node in enumerate(node2vec_model.node_names)}
            tag_to_embeddings = {node_id_to_tag[i]: tags_vectors_full[i].detach().numpy() for i in
                                 range(len(tags_vectors_full))}
            sentences = train_df['all_tags'].tolist()
            tags_vectors = np.array([np.mean([tag_to_embeddings[word] for word in tag if word in tag_to_embeddings],
                                              axis=0) if any(word in tag_to_embeddings for word in tag) else np.zeros(
                node2vec_model.embedding_dim) for tag in sentences])
            tags_model = (node2vec_model, tag_to_embeddings)
        else:
            raise ValueError("Модель Node2Vec не предоставлена для оценки.")
    elif tags_method == 'tfidf':
        tags_vectors, tags_vectorizer = vectorize_tags(train_df, method=tags_method)
    elif tags_method == 'multilabel':
        tags_vectors, tags_mlb = vectorize_tags(train_df, method=tags_method, multilabel_params=multilabel_params)
        tags_vectorizer = tags_mlb
    elif tags_method == 'word2vec':
        tags_vectors, tags_model = vectorize_tags(train_df, method=tags_method, word2vec_params=word2vec_params_tags)
    elif tags_method == 'fasttext':
        tags_vectors, tags_model = vectorize_tags(train_df, method=tags_method, fasttext_params=fasttext_params_tags)
    else:
        tags_vectors, tags_model = vectorize_tags(train_df, method=tags_method)
    tags_end_time = time.time()
    print(f"Векторизация тегов завершена за {tags_end_time - tags_start_time:.2f} секунд")

    # Векторизация описаний
    desc_start_time = time.time()
    desc_model = None
    desc_vectorizer = None
    if desc_method == 'tfidf':
        desc_vectors, desc_vectorizer = vectorize_descriptions(train_df, method=desc_method)
    elif desc_method == 'word2vec':
        desc_vectors, desc_model = vectorize_descriptions(train_df, method=desc_method,
                                                           word2vec_params=word2vec_params_desc)
    elif desc_method == 'sentence_bert':
        desc_vectors, desc_model = vectorize_descriptions(train_df, method=desc_method,
                                                           sentence_bert_model=sentence_bert_model)
    elif desc_method == 'lda':
        desc_vectors, (desc_model, desc_vectorizer) = vectorize_descriptions(train_df, method=desc_method,
                                                                             lda_params=lda_params)
    elif desc_method == 'nmf':
        desc_vectors, (desc_model, desc_vectorizer) = vectorize_descriptions(train_df, method=desc_method,
                                                                             nmf_params=nmf_params)
    else:
        desc_vectors, desc_model = vectorize_descriptions(train_df, method=desc_method)
    desc_end_time = time.time()
    print(f"Векторизация описаний завершена за {desc_end_time - desc_start_time:.2f} секунд")

    # Объединение векторов
    combine_start_time = time.time()
    combined_vectors = np.hstack([owners_vectors_weighted, tags_vectors, desc_vectors])

    pca = None
    # Ограничение размерности объединенных векторов с использованием PCA
    if combined_vectors.shape[1] > 10000:  # Применяется, только если размерность превышает 10000
        n_components = min(10000, combined_vectors.shape[0])  # Гарантируем, что n_components не больше числа образцов
        print(
            f"Уменьшение размерности объединенного вектора с {combined_vectors.shape[1]} до {n_components} с помощью PCA...")
        pca = PCA(n_components=n_components)
        combined_vectors = pca.fit_transform(combined_vectors)

    # Масштабирование объединенных векторов
    scaler = MinMaxScaler()
    scaler.fit(combined_vectors)
    combined_vectors = scaler.transform(combined_vectors)
    combine_end_time = time.time()
    print(f"Объединение векторов завершено за {combine_end_time - combine_start_time:.2f} секунд")

    # Вычисление матрицы схожести
    similarity_start_time = time.time()
    # Ограничение расчета матрицы схожести для больших наборов данных
    if combined_vectors.shape[0] > 2000:  # Произвольное пороговое значение
        print("Пропуск расчета полной матрицы схожести для большого набора данных.")
        similarity_matrix = None
    else:
        similarity_matrix = cosine_similarity(combined_vectors)
    similarity_end_time = time.time()
    print(f"Расчет схожести завершен за {similarity_end_time - similarity_start_time:.2f} секунд")

    end_time = time.time()
    print(f"Выполнение evaluate_vectors завершено за {end_time - start_time:.2f} секунд")
    return combined_vectors, similarity_matrix, scaler, (tags_model, desc_model, tags_vectorizer, desc_vectorizer), pca

In [18]:
def test_vector_combination(test_df, train_df, owners_method, tags_method, desc_method, tags_model=None, desc_model=None,
                            scaler=None, tags_vectorizer=None, desc_vectorizer=None, node2vec_model=None,
                            test_game_name=None, pca=None, sentence_bert_model=None):
    """
    Тестирует комбинацию векторизации и оценивает схожесть.

    Args:
        test_df (pd.DataFrame): Тестовый DataFrame.
        train_df (pd.DataFrame): Обучающий DataFrame.
        owners_method (str): Метод векторизации владельцев.
        tags_method (str): Метод векторизации тегов.
        desc_method (str): Метод векторизации описаний.
        tags_model: Модель векторизации тегов.
        desc_model: Модель векторизации описаний.
        scaler: Скалер для масштабирования векторов.
        tags_vectorizer: Векторизатор тегов.
        desc_vectorizer: Векторизатор описаний.
        node2vec_model: Обученная модель Node2Vec.
        test_game_name (str): Название тестовой игры.
        pca: Модель PCA для уменьшения размерности.
        sentence_bert_model: Модель Sentence-BERT.

    Returns:
        dict: Результаты тестирования (методы, название тестовой игры, топ-5 похожих игр и их оценки).
    """
    print(
        f"--- Тестирование комбинации векторов: Владельцы={owners_method}, Теги={tags_method}, Описания={desc_method} ---")
    start_time = time.time()

    # Векторизация обучающих данных (используются обученные ранее модели)
    train_owners_vectors = vectorize_owners(train_df, method=owners_method)
    if tags_method == 'tfidf':
        train_tags_vectors = tags_vectorizer.transform(train_df['all_tags'].apply(lambda x: ' '.join(x))).toarray()
    elif tags_method == 'multilabel':
        train_tags_vectors = tags_vectorizer.transform(train_df['all_tags'])
    elif tags_method == 'word2vec':
        sentences = train_df['all_tags'].tolist()
        train_tags_vectors = np.array([np.mean([tags_model.wv[word] for word in tag if word in tags_model.wv], axis=0)
                                       if any(word in tags_model.wv for word in tag) else np.zeros(
            tags_model.vector_size) for tag in sentences])
    elif tags_method == 'fasttext':
        sentences = train_df['all_tags'].tolist()
        train_tags_vectors = np.array([np.mean([tags_model.wv[word] for word in tag if word in tags_model.wv], axis=0)
                                       if any(word in tags_model.wv for word in tag) else np.zeros(
            tags_model.vector_size) for tag in sentences])
    elif tags_method == 'node2vec':
        if node2vec_model is not None:
            with torch.no_grad():
                tags_vectors_full = node2vec_model().cpu()
            node_id_to_tag = {i: node for i, node in enumerate(node2vec_model.node_names)}
            tag_to_embeddings = {node_id_to_tag[i]: tags_vectors_full[i].detach().numpy() for i in
                                 range(len(tags_vectors_full))}
            sentences = train_df['all_tags'].tolist()
            train_tags_vectors = np.array([np.mean([tag_to_embeddings[word] for word in tag if word in tag_to_embeddings],
                                              axis=0) if any(word in tag_to_embeddings for word in tag) else np.zeros(
                node2vec_model.embedding_dim) for tag in sentences])
        else:
            raise ValueError("Модель Node2Vec не предоставлена для тестирования.")
    else:
        raise ValueError(f"Неизвестный метод векторизации тегов: {tags_method}")

    if desc_method == 'tfidf':
        train_desc_vectors = desc_vectorizer.transform(train_df['short_description_clean']).toarray()
    elif desc_method == 'word2vec':
        sentences = train_df['short_description_clean'].apply(lambda x: x.split()).tolist()
        train_desc_vectors = np.array(
            [np.mean([desc_model.wv[word] for word in desc if word in desc_model.wv], axis=0) if any(
                word in desc_model.wv for word in desc) else np.zeros(desc_model.vector_size) for desc in sentences])
    elif desc_method == 'sentence_bert':
        train_desc_vectors = sentence_bert_model.encode(train_df['short_description_clean'].tolist(), device=device)
    elif desc_method == 'lda':
        train_desc_vectors = desc_model.transform(desc_vectorizer.transform(train_df['short_description_clean']))
    elif desc_method == 'nmf':
        train_desc_vectors = desc_model.transform(desc_vectorizer.transform(train_df['short_description_clean']))
    else:
        raise ValueError(f"Неизвестный метод векторизации описаний: {desc_method}")

    train_combined_vectors = np.hstack([train_owners_vectors, train_tags_vectors, train_desc_vectors])

    # Выбор тестовой игры
    if test_game_name:
        test_game = test_df[test_df['name'] == test_game_name]
    else:
        test_game = test_df.sample(1)

    # Векторизация тестовых данных
    test_owners_vectors = vectorize_owners(test_game, method=owners_method)
    if tags_method == 'tfidf':
        test_tags_vectors = tags_vectorizer.transform(test_game['all_tags'].apply(lambda x: ' '.join(x))).toarray()
    elif tags_method == 'multilabel':
        test_tags_vectors = tags_vectorizer.transform(test_game['all_tags'])
    elif tags_method == 'word2vec':
        sentences = test_game['all_tags'].tolist()
        test_tags_vectors = np.array([np.mean([tags_model.wv[word] for word in tag if word in tags_model.wv], axis=0)
                                      if any(word in tags_model.wv for word in tag) else np.zeros(
            tags_model.vector_size) for tag in sentences])
    elif tags_method == 'fasttext':
        sentences = test_game['all_tags'].tolist()
        test_tags_vectors = np.array([np.mean([tags_model.wv[word] for word in tag if word in tags_model.wv], axis=0)
                                      if any(word in tags_model.wv for word in tag) else np.zeros(
            tags_model.vector_size) for tag in sentences])
    elif tags_method == 'node2vec':
        if node2vec_model is not None:
            with torch.no_grad():
                tags_vectors_full = node2vec_model().cpu()
            node_id_to_tag = {i: node for i, node in enumerate(node2vec_model.node_names)}
            tag_to_embeddings = {node_id_to_tag[i]: tags_vectors_full[i].detach().numpy() for i in
                                 range(len(tags_vectors_full))}
            sentences = test_game['all_tags'].tolist()
            test_tags_vectors = np.array([np.mean([tag_to_embeddings[word] for word in tag if word in tag_to_embeddings],
                                             axis=0) if any(word in tag_to_embeddings for word in tag) else np.zeros(
                node2vec_model.embedding_dim) for tag in sentences])
        else:
            raise ValueError("Модель Node2Vec не предоставлена для тестирования.")
    else:
        raise ValueError(f"Неизвестный метод векторизации тегов: {tags_method}")

    if desc_method == 'tfidf':
        test_desc_vectors = desc_vectorizer.transform(test_game['short_description_clean']).toarray()
    elif desc_method == 'word2vec':
        sentences = test_game['short_description_clean'].apply(lambda x: x.split()).tolist()
        test_desc_vectors = np.array(
            [np.mean([desc_model.wv[word] for word in desc if word in desc_model.wv], axis=0) if any(
                word in desc_model.wv for word in desc) else np.zeros(desc_model.vector_size) for desc in sentences])
    elif desc_method == 'sentence_bert':
        test_desc_vectors = sentence_bert_model.encode(test_game['short_description_clean'].tolist(), device=device)
    elif desc_method == 'lda':
        test_desc_vectors = desc_model.transform(desc_vectorizer.transform(test_game['short_description_clean']))
    elif desc_method == 'nmf':
        test_desc_vectors = desc_model.transform(desc_vectorizer.transform(test_game['short_description_clean']))
    else:
        raise ValueError(f"Неизвестный метод векторизации описаний: {desc_method}")
    
    test_combined_vectors = np.hstack([test_owners_vectors, test_tags_vectors, test_desc_vectors])

    # Применение PCA к тестовым векторам, если PCA использовался при обучении
    if pca is not None:
        if train_combined_vectors.shape[1] > 10000:
            test_combined_vectors = pca.transform(test_combined_vectors)

    # Применение скалера к тестовым векторам, если скалер использовался при обучении
    if scaler is not None:
        # Проверка на случай, если размерность тестовых и обучающих векторов отличается после PCA
        if train_combined_vectors.shape[1] != test_combined_vectors.shape[1]:
            min_features = min(train_combined_vectors.shape[1], test_combined_vectors.shape[1])
            train_combined_vectors = train_combined_vectors[:, :min_features]
            test_combined_vectors = test_combined_vectors[:, :min_features]
        train_combined_vectors = scaler.transform(train_combined_vectors)
        test_combined_vectors = scaler.transform(test_combined_vectors)

    # Оценка качества на тестовых данных (вывод 5 наиболее похожих игр)
    similarities = cosine_similarity(test_combined_vectors.reshape(1, -1), train_combined_vectors)
    most_similar_indices = np.argsort(similarities[0])[-5:][::-1]

    # Получаем имя игры напрямую из test_game DataFrame
    test_game_name_value = test_game['name'].iloc[0]

    top_similar_games = [train_df['name'].iloc[idx] for idx in most_similar_indices]
    similarity_scores = [similarities[0][idx] for idx in most_similar_indices]

    end_time = time.time()
    print(f"Выполнение test_vector_combination завершено за {end_time - start_time:.2f} секунд")
    return {
        'owners_method': owners_method,
        'tags_method': tags_method,
        'desc_method': desc_method,
        'test_game_name': test_game_name_value,
        'top_similar_game_1': top_similar_games[0],
        'similarity_score_1': similarity_scores[0],
        'top_similar_game_2': top_similar_games[1],
        'similarity_score_2': similarity_scores[1],
        'top_similar_game_3': top_similar_games[2],
        'similarity_score_3': similarity_scores[2],
        'top_similar_game_4': top_similar_games[3],
        'similarity_score_4': similarity_scores[3],
        'top_similar_game_5': top_similar_games[4],
        'similarity_score_5': similarity_scores[4],
    }

In [19]:
vector_combinations = [
    {'owners': 'log_scale', 'tags': 'multilabel', 'desc': 'nmf'},
    {'owners': 'log_scale', 'tags': 'multilabel', 'desc': 'lda'},
]

In [20]:
if any(combo['tags'] == 'node2vec' for combo in vector_combinations):
    print("Обучение модели Node2Vec...")
    start_time = time.time()
    G = nx.Graph()
    for tags_list in train_df['all_tags']:
        for i in range(len(tags_list)):
            for j in range(i + 1, len(tags_list)):
                tag1 = tags_list[i]
                tag2 = tags_list[j]
                if G.has_edge(tag1, tag2):
                    G[tag1][tag2]['weight'] += 1
                else:
                    G.add_edge(tag1, tag2, weight=1)

    # Добавление ребер на основе общих соседей
    for tag1 in list(G.nodes()): # Проходим по копии, чтобы избежать проблем с изменением размера графа
        for tag2 in list(G.nodes()):
            if tag1 != tag2 and not G.has_edge(tag1, tag2):
                common_neighbors = list(nx.common_neighbors(G, tag1, tag2))
                if common_neighbors:
                    weight = len(common_neighbors)
                    G.add_edge(tag1, tag2, weight=weight)

    data = from_networkx(G).to(device)
    torch.manual_seed(42)
    embedding_dim = 120
    walk_length = 20
    context_size = 8
    walks_per_node = 20
    num_negative_samples = 10
    p = 1.0
    q = 0.8
    num_epochs = 40
    learning_rate = 0.01

    node2vec_model = Node2Vec(data.edge_index, embedding_dim=embedding_dim, walk_length=walk_length,
                               context_size=context_size, walks_per_node=walks_per_node,
                               num_negative_samples=num_negative_samples, p=p, q=q).to(device)
    loader = node2vec_model.loader(batch_size=256, shuffle=True)
    optimizer = optim.Adam(node2vec_model.parameters(), lr=learning_rate)

    losses = []
    for epoch in range(num_epochs):
        node2vec_model.train()
        total_loss = 0
        for pos_rw, neg_rw in loader:
            pos_rw = pos_rw.to(device)
            neg_rw = neg_rw.to(device)
            optimizer.zero_grad()
            loss = node2vec_model.loss(pos_rw, neg_rw)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        epoch_loss = total_loss / len(loader)
        losses.append(epoch_loss)
        print(f'Эпоха Node2Vec {epoch + 1}: Loss = {epoch_loss}')

    # Визуализация кривой обучения
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, num_epochs + 1), losses, marker='o')
    plt.title('Кривая обучения Node2Vec')
    plt.xlabel('Эпоха')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()

    node2vec_model.node_names = list(G.nodes())
    end_time = time.time()
    print(f"Модель Node2Vec обучена за {end_time - start_time:.2f} секунд")

    # Визуализация эмбеддингов
    if len(G.nodes()) <= 1000:
        print("Визуализация эмбеддингов Node2Vec...")
        with torch.no_grad():
            node_embeddings = node2vec_model().cpu().numpy()

        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(G.nodes()) - 1), n_iter=300, early_exaggeration=20)
        embeddings_reduced = tsne.fit_transform(node_embeddings)

        tags = list(G.nodes())
        plt.figure(figsize=(16, 12))
        sns.scatterplot(x=embeddings_reduced[:, 0], y=embeddings_reduced[:, 1], alpha=0.7, s=100)

        for i, tag in enumerate(tags):
            plt.annotate(str(i),
                         xy=(embeddings_reduced[i, 0], embeddings_reduced[i, 1]),
                         xytext=(5, 5),
                         textcoords='offset points',
                         ha='center',
                         va='bottom',
                         fontsize=12)

        plt.title('Визуализация эмбеддингов Node2Vec (t-SNE)', fontsize=16)
        plt.xlabel('Измерение t-SNE 1', fontsize=14)
        plt.ylabel('Измерение t-SNE 2', fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

        print("\nСписок тегов:")
        for i, tag in enumerate(tags):
            print(f"{i}: {tag}")

    else:
        print("Слишком много тегов для эффективной визуализации эмбеддингов.")

else:
    node2vec_model = None

In [21]:
manual_test_game_name = "Baldur's Gate 3"
results = [] # Список для хранения результатов

In [22]:
sentence_bert_model = SentenceTransformer('all-mpnet-base-v2').to(device)

In [23]:
top_10_test_games = test_df.sort_values(by='estimated_owners', ascending=False).head(10)

In [24]:
for combo in vector_combinations:
    print(f"Тестирование комбинации: {combo}")

    # Извлекаем параметры векторизации из combo
    owners_method = combo['owners']
    tags_method = combo['tags']
    desc_method = combo['desc']
    word2vec_params_tags = combo.get('word2vec_params_tags')
    fasttext_params_tags = combo.get('fasttext_params_tags')
    multilabel_params = combo.get('multilabel_params')
    word2vec_params_desc = combo.get('word2vec_params_desc')
    lda_params = combo.get('lda_params')
    nmf_params = combo.get('nmf_params')
    owners_weight_factor = combo.get('owners_weight_factor', 1.0)

    # Обучаем модели векторизации на обучающей выборке
    combined_vectors, similarity_matrix, scaler, (tags_model, desc_model, tags_vectorizer, desc_vectorizer), pca = evaluate_vectors(
        train_df,
        owners_method=owners_method,
        tags_method=tags_method,
        desc_method=desc_method,
        node2vec_model=node2vec_model,
        owners_weight_factor=owners_weight_factor,
        word2vec_params_tags=word2vec_params_tags,
        fasttext_params_tags=fasttext_params_tags,
        multilabel_params=multilabel_params,
        word2vec_params_desc=word2vec_params_desc,
        lda_params=lda_params,
        nmf_params=nmf_params,
        sentence_bert_model=sentence_bert_model
    )

    # Итерируемся по топ-10 играм из test_df
    for index, test_game_row in top_10_test_games.iterrows():
        test_game_name = test_game_row['name']
        print(f"Тестирование с игрой из топ-10: '{test_game_name}'")
        result = test_vector_combination(
            test_df, train_df,
            owners_method=owners_method,
            tags_method=tags_method,
            desc_method=desc_method,
            tags_model=tags_model,
            desc_model=desc_model,
            scaler=scaler,
            tags_vectorizer=tags_vectorizer,
            desc_vectorizer=desc_vectorizer,
            node2vec_model=node2vec_model,
            test_game_name=test_game_name,
            pca=pca,
            sentence_bert_model=sentence_bert_model
        )
        results.append(result)

Тестирование комбинации: {'owners': 'log_scale', 'tags': 'multilabel', 'desc': 'nmf'}
Начало evaluate_vectors с владельцами: log_scale, тегами: multilabel, описаниями: nmf
Начало векторизации владельцев с использованием метода: log_scale
Применен весовой коэффициент 1.0 к вектору владельцев.
Векторизация владельцев завершена за 0.00 секунд
Начало векторизации тегов с использованием метода: multilabel
Параметры MultiLabelBinarizer: {'sparse_output': False}
Завершение векторизации тегов с использованием метода: multilabel
Векторизация тегов завершена за 0.41 секунд
Начало векторизации описаний с использованием метода: nmf
Параметры NMF: {'n_components': 10, 'init': 'nndsvdar', 'solver': 'mu', 'beta_loss': 'frobenius', 'random_state': 0}
Завершение векторизации описаний с использованием метода: nmf
Векторизация описаний завершена за 3.01 секунд
Объединение векторов завершено за 0.18 секунд
Пропуск расчета полной матрицы схожести для большого набора данных.
Расчет схожести завершен за 0.00

In [25]:
print("\nТаблица лучших результатов:")
results_df = pd.DataFrame(results)
results_df


Таблица лучших результатов:


,owners_method,tags_method,desc_method,test_game_name,top_similar_game_1,similarity_score_1,top_similar_game_2,similarity_score_2,top_similar_game_3,similarity_score_3,top_similar_game_4,similarity_score_4,top_similar_game_5,similarity_score_5
0,log_scale,multilabel,nmf,Black Squad,METRO CONFLICT: THE ORIGIN,0.779276,BATTALION 1944,0.729126,Battlefield 4™,0.715391,Magnificent 5,0.708823,Battlefield 3™,0.705153
1,log_scale,multilabel,nmf,Street Warriors Online,Drone Fighters,0.709003,Scribbled Arena,0.687157,Sniper Fury,0.666516,Wheel Riders Online OBT,0.662142,GetMeBro!,0.660462
2,log_scale,multilabel,nmf,World of Warships,World of Tanks,0.854986,Enlisted: Reinforced,0.690843,World of Warplanes,0.680448,War Thunder,0.674935,WWII Online,0.662009
3,log_scale,multilabel,nmf,BioShock Infinite,BioShock™ 2 Remastered,0.689201,BioShock® 2,0.689069,Wolfenstein II: The New Colossus,0.674355,RoboCop: Rogue City,0.637408,Deus Ex: Human Revolution - Director's Cut,0.636759
4,log_scale,multilabel,nmf,Deathmatch Classic,Quake III Arena,0.751294,Team Fortress Classic,0.733756,Ultimate Doom,0.695989,Unreal Tournament: Game of the Year Edition,0.677561,QUAKE II,0.659948
5,log_scale,multilabel,nmf,Red Dead Redemption 2,Red Dead Online,0.795121,Grand Theft Auto V,0.692896,Desert Of The Dead,0.685571,Call of Duty®: Modern Warfare® II,0.673395,Call of Duty®: Modern Warfare® II,0.655600
6,log_scale,multilabel,nmf,Half-Life,Half-Life: Opposing Force,0.765076,Half-Life: Source,0.716980,Quake III Arena,0.673501,Turok 2: Seeds of Evil,0.673216,Half-Life: Blue Shift,0.673020
7,log_scale,multilabel,nmf,Cities: Skylines,Smart City Plan,0.746300,Planet Zoo,0.700812,Cities XXL,0.697426,Zoo Constructor,0.688399,Project Hospital,0.687457
8,log_scale,multilabel,nmf,Day of Defeat,Tom Clancy's Ghost Recon®,0.734497,Brothers in Arms: Road to Hill 30™,0.727120,Red Orchestra: Ostfront 41-45,0.709054,Call of Duty: United Offensive,0.686379,Call of Duty®,0.678756
9,log_scale,multilabel,nmf,BattleBlock Theater®,Move or Die,0.769514,Castle Crashers®,0.760296,Bloody Trapland 2: Curiosity,0.757250,Trine Enchanted Edition,0.750338,No Time To Explain Remastered,0.747039
